# Import necessary libraries

In [16]:
import configparser
import os
from dotenv import load_dotenv

In [17]:
config = configparser.ConfigParser()
config.read('config.env')

['config.env']

In [18]:
load_dotenv()

False

# Get Connection Details   

In [19]:
conn = snowflake.connector.connect(
    user=config['snowflake']['user'],     
    password=config['snowflake']['password'], 
    account=config['snowflake']['account'],    
    warehouse=config['snowflake']['warehouse'],  
    database=config['snowflake']['database'],    
    schema=config['snowflake']['schema'],        
    client_session_keep_alive=True
)

# Establish snowflake connection

In [20]:
print("Connecting to Snowflake...")
cursor = conn.cursor()

Connecting to Snowflake...


# Queries to answer some business questions

In [21]:
# Query 1: Most popular transport types
most_popular_types_query = """
SELECT journey_type,
    SUM(journeys_millions)/1000000 as total_journeys_millions
FROM KBELE_LONDON.PUBLIC.JOURNEYS
GROUP BY journey_type
ORDER BY total_journeys_millions DESC;
"""

# Query 2: Emirates Airline popularity by month and year
emirates_query = """
SELECT month,
    year,
    ROUND(journeys_millions/1000000, 2) as rounded_journeys_millions
FROM KBELE_LONDON.PUBLIC.JOURNEYS
WHERE journey_type = 'Emirates Airline'
AND journeys_millions IS NOT NULL
ORDER BY rounded_journeys_millions DESC
LIMIT 5;
"""

# Query 3: Least popular years for Underground & DLR
underground_query = """
SELECT year,
    journey_type,
    SUM(journeys_millions)/1000000 as total_journeys_millions
FROM KBELE_LONDON.PUBLIC.JOURNEYS
WHERE journey_type = 'Underground & DLR'
GROUP BY year, journey_type
ORDER BY total_journeys_millions ASC
LIMIT 5;
"""

# Execute first query

In [22]:
try:
    # query
    # query = "SELECT * FROM KBELE_LONDON.PUBLIC.JOURNEYS LIMIT 10;"  
    # cursor.execute(query)

    # # Fetch the results
    # results = cursor.fetchall()

    # # Print the results
    # print("Query Results:")
    # for row in results:
    #     print(row)
    print("Connecting to Snowflake...")
    cursor = conn.cursor()
    # Execute and fetch results for each query
    print("\nMost Popular Transport Types:")
    cursor.execute(most_popular_types_query)
    results1 = cursor.fetchall()
    df_popular_types = pd.DataFrame(results1, columns=['journey_type', 'total_journeys_millions'])
    print(df_popular_types)
        
    # print("\nTop 5 Months for Emirates Airline:")
    # cursor.execute(emirates_query)
    # results2 = cursor.fetchall()
    # for row in results2:
    #     print(row)
        
    # print("\nLeast Popular Years for Underground & DLR:")
    # cursor.execute(underground_query)
    # results3 = cursor.fetchall()
    # for row in results3:

except Exception as e:
    print(f"Error: {e}")
finally:
    cursor.close()
    conn.close()

Connecting to Snowflake...

Most Popular Transport Types:
        journey_type total_journeys_millions
0                Bus          0.024905193947
1  Underground & DLR          0.015020466544
2         Overground          0.001666845666
3           TfL Rail          0.000411313421
4               Tram          0.000314689875
5   Emirates Airline          0.000014583718
